In [6]:
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import random
import tensorflow as tf
from tensorflow.python.keras import backend as k
import os
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sbn
import pandas as pd
from sklearn.metrics import roc_curve, auc


CLASS_NAMES = os.listdir("Picked_data/")
CLASS_NAMES.remove('.DS_Store')

In [7]:
def resetRandom():
    seed = 1
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.compat.v1.random.set_random_seed(seed)
    session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
    tf.compat.v1.set_random_seed(seed)
    sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
    k.set_session(sess)

In [8]:
def makedirs(*args):
    for arg in args:
        os.makedirs(arg, exist_ok=True)

In [9]:
def plot_conf_matrix(conf_mat, fe, cls, labels, path):
    fig, ax = plt.subplots(figsize=(25,15))
    sbn.heatmap(conf_mat, annot=True, annot_kws={"size": 10}, linewidths=0.5, fmt='d',
                yticklabels=labels, xticklabels=labels, cmap='Blues')
    plt.xlabel('Actual Class', labelpad=15)
    plt.ylabel('Predicted Class', labelpad=15)
    labels = ["Feature = {0}".format(fe), "Classifier = {0}".format(cls)]
    trans = ax.get_xaxis_transform()
    ax.annotate('\n'.join(labels), xy=(12, 0.5), xycoords=trans, size=17, ha='left', va='baseline',
                bbox=dict(boxstyle='round', fc='w'))
    plt.savefig(path, bbox_inches="tight")
    plt.close()
    plt.clf()
  


In [10]:
def plot_roc_curve(y, pred, fe, cls, results_dir):
    n_classes = len(CLASS_NAMES)
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(np.array(pd.get_dummies(y))[:, i], np.array(pd.get_dummies(pred))[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(n_classes):
        mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])
    mean_tpr /= n_classes
    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])
    fig, ax = plt.subplots(figsize=(25, 15))
    plt.plot(fpr['macro'], tpr['macro'], color='orange', label='AUC = {0}'.format(roc_auc['macro']))
    plt.plot([0, 1], [np.min(tpr['macro']), 1], color='darkblue', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend()
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    labels = ["Feature = {0}".format(fe), "Classifier = {0}".format(cls)]
    trans = ax.get_xaxis_transform()
    ax.annotate('\n'.join(labels), xy=(1.06, 0.5), xycoords=trans, size=14, ha='left', va='baseline',
                bbox=dict(boxstyle='round', fc='w'))
    path = os.path.join(results_dir, '{0}_{1}_roc_curve.png'.format(fe, cls))
    plt.savefig(path, bbox_inches="tight")
    plt.clf()
    plt.close()
